In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import random   

np.random.seed(42)
random.seed(42)

desire = 'Eat'
test_data_path = f'data/test/{desire}.csv'
test_df = pd.read_csv(test_data_path, index_col=False)
test_df.head(2)

,person_id,travel_purpose,target_amenity,mode,distance_miles,duration_minutes,age,individual_income,household_size,family_structure,vehicles,cypher
0,5.570035e+18,Eat,F&B Eatery/Drinking Places,Walking,0.0,0.0,29,53672,3_person,family_single,1,\n MATCH (p:Person)\n WHERE (p.a...
1,7.600486e+18,Eat,F&B Eatery/Snack and Nonalcoholic Beverage Bars,Walking,0.0,0.0,26,0,3_person,nonfamily_single,0,\n MATCH (p:Person)\n WHERE (p.a...


In [ ]:
from model import ChoiceModel
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd

num_sample = 900
num_threads = 3

data = []

choice_model = ChoiceModel(
    data_dir='data', desire=desire, sample_num=num_sample, skip_init=True)

def process_row(row):
    profile = row[['person_id', 'age', 'individual_income',
                    'household_size', 'family_structure', 'vehicles', 'cypher']]
    profile = profile.to_dict()
    response = choice_model.infer(
        profile=profile, mode='experiment',in_parallel=True)
    return response

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_row, row) for idx, row in test_df.iterrows()]
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            response = future.result()
            if response is not None:
                print('get response', response)
            data.append(response)
        except Exception as e:
            print(f"Error get response: {e}")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# for idx, row in test_df.iterrows():
#     process_row(row)
#     break

profile A 29 year old person, living in a family_single family with 3_person members. The person has 1 vehicles and an annual income of 53672 dollars.
most similar profile [{'income': 158190, 'household_size': '4_person', 'name': 'Person', 'description': 'A 29 year old person, living in a nonfamily_single family with 4_person members. The person has 1 vehicles and an annual income of 158190 dollars.', 'vehicles': '1', 'id': 'Person_8.369226427629497e+18', 'embedding': [-0.14670468866825104, 1.0552116632461548, -3.1932177543640137, -0.8326756954193115, -0.7992340326309204, 0.2598385810852051, 0.32707491517066956, 0.0014591515064239502, -0.14490671455860138, -0.4482768476009369, -1.1131768226623535, -0.8381162881851196, 0.19708514213562012, 0.3134186565876007, 0.09706217050552368, 0.528935432434082, -0.08284933120012283, 0.046635791659355164, 0.07634662091732025, 0.6465086340904236, -0.7088171243667603, 0.1557469218969345, -1.358534812927246, 0.7198455333709717, 2.69069242477417, 1.07500

In [ ]:
data_df = pd.DataFrame(data, columns=['person_id', 'profile', 'top_k', 'desire', 'city', 'cypher',
                                      'amenity_recommendation', 'amenity_llm_choice', 'amenity_final_choice',
                                      'mode_recommendation', 'mode_llm_choice', 'mode_final_choice'])